## 1. 예측 모델에 대한 개발 설명 

주식보유기간 예측은 기본적으로 회귀분석이라 회귀모델을사용했다

hold_d의 날짜 값을 예측하는 것으로 계좌 ID와 종목데이터를 hist 파일에서 찾아 train에 hist값을 설정하고 test의 hold를 예측하는 것을 목표로 하였다.

train에 hist 값을 설정하기 위해 train의 hist값은 베이스 코드를 참고하여 시행착오법을 활용하였다.

 train[‘hold_d’]*0.6을 참고하여 맨처음 train으로 test를 hold_d를 예측한것을 기반으로 train["act_id"].groupby 한걸로 merge 해서 hold_d/hist_d 를 산출해서 분석했다 

### 1. 분석 배경 및 과정

dacon에서 제시해준 Hold_d(보유기간) – hist_d(‘20년 12월 31일까지의 최근 보유기간) ≤ 146
을 기준으로  1 ≤ Hold_d(보유기간) – hist_d < 147 으로 생각했다

 train[‘hold_d’]*0.6을 참고하여 'hist_d'는 얼마나 보유했는데 
 앞으로 얼마나더 보유할지 예측하는상황인데.
 애시당초에 train에 hist_d가 없이 임의로 
 train["hold_d"]*0.6 = train["hist_d"]로 학습하는건 이상하다고 
 판단했고 dacon baseline이 제공해준값이아닌 다른값을 부여해
 train을 학습해 test에 predict시킨값을 train["act_id"].groupby 한걸로 merge 해서 hold_d/hist_d 비율을 계산했고 비율을 train["hold_d"]에 곱해 train["hist_d"]로 부여했다


### 2. 채택된 변수에 대한 설명

#### cus_info

1. cus_info.csv 
컬럼 ivs_icn_cd 투자성향 컬럼은 
계좌 가입시 설문조사로 결정되는 값으로 주관적인 판단이며
print(cus[cus['ivs_icn_cd'] == 99].shape[0] / cus.shape[0]) == 0.3687
미정의(99) 또한 36%의 비율로 상당히 많다.

2. mrz_pdt_tp_sgm_cd 변수는 특정 값(02:국내주식)의 비율이 96.59%로 특정값에 몰려있다.
즉, 종속변수에 영향을 미치는 변수라고 볼 수 없다.

    (cus[cus['mrz_pdt_tp_sgm_cd']==2]['mrz_pdt_tp_sgm_cd'].count() / cus['mrz_pdt_tp_sgm_cd'].value_counts().sum() ==0.9659 )

3. lsg_sgm_cd의 값 중 5, 6에서 다음과 같은 오점이 발견되었다.
주최측에서 배분한 데이터스키마csv의 변수설명에 따르면 
lsg_sgmcd 변수 중 06값[가족성숙기주부 (40-59세 & 주부 & 여자 )]은 성별이 여자이어야 하나, 데이터 상으로 남성이 포함되는 것으로 나타났다.
또한 이 컬럼은 sex_dit_cd , cus_age_stn_cd가 내포한 의미와 똑같기 때문에 학습에 포함시키지 않았다 

   pd.crosstab(cus["sex_dit_cd"], cus["lsg_sgm_cd"])

4. 주최측에서 배분한 데이터스키마csv의 변수설명에 따라 tco_cus_grd_cd변수의 결측치 처리와 미정의된 데이터의 재정의를 위하여 
고객등급(tco_cus_grd_cd)과 고객 자산구간 변수(cus_aet_stn_cd)를 연계하여 다음과 같이 처리하였다.
고객등급 결측치(99)를 고객 자산구간별으로 분배
cus['tco_cus_grd_cd'][(cus['tco_cus_grd_cd'] == 99) & (cus['cus_aet_stn_cd'] == 1)] = 3 # 고객 자산 1억 이상, 3억 미만
cus['tco_cus_grd_cd'][(cus['tco_cus_grd_cd'] == 99) & (cus['cus_aet_stn_cd'] == 2)] = 3 # 고객 자산 1억 이상, 3억 미만
고객등급 미정의(9)를 고객 자산구간별으로 분배
cus['tco_cus_grd_cd'][(cus['tco_cus_grd_cd'] == 9) & (cus['cus_aet_stn_cd'] == 1)] = 9 # 고객 자산 1천 미만
cus['tco_cus_grd_cd'][(cus['tco_cus_grd_cd'] == 9) & (cus['cus_aet_stn_cd'] == 2)] = 5 # 고객 자산 1천 이상, 3천 미만
cus['tco_cus_grd_cd'][(cus['tco_cus_grd_cd'] == 9) & (cus['cus_aet_stn_cd'] == 3)] = 4 # 고객 자산 3천 이상, 1억 미만
cus['tco_cus_grd_cd'][(cus['tco_cus_grd_cd'] == 9) & (cus['cus_aet_stn_cd'] == 4)] = 4 # 고객 자산 3천 이상, 1억 미만
cus['tco_cus_grd_cd'][(cus['tco_cus_grd_cd'] == 9) & (cus['cus_aet_stn_cd'] == 5)] = 3 # 고객 자산 1억 이상, 3억 미만
서비스 등급 미정의(9)는 1천만원 미만의 자산을 보유한 고객으로 보아 고객등급 6으로 설정
cus['tco_cus_grd_cd'][(cus['tco_cus_grd_cd'] == 9)] = 6

#### iem_info

1. mkt_pr_tal_scl_tp_cd 시총규모
0 대형주, 1 중형주, 3 소형주, 99(기타) 로 분류되어있다.
99는 결측값으로 판단하여 기타(99) -> 3

    iem['mkt_pr_tal_scl_tp_cd'][iem['mkt_pr_tal_scl_tp_cd'] == 99] = 3 최빈값인 3으로 부여하였다

2. iem.stk_dit_cd 시장구분
시장구분 1 코스피, 2 코스닥, 99 기타로 분류되어있다 값이 99라면 학습에 방해될수가있어 99(기타)를 3으로 부여하였다 

   iem['stk_dit_cd'][iem['stk_dit_cd'] == 99] = 3


#### stk_bnc_hist

1. hist.stk_p : 매매 평균단가 / 파생변수
hist['stk_p'] = hist['tot_aet_amt'] / hist['bnc_qty'] 잔고금액 / 잔고수량의 값으로 종목의 매수매도의 평균단가 파생변수 생성

### C. 모델링 진행 및 중요한 가정들

1. print("min", train["hold_d"].min(), "max", train["hold_d"].max())
학습하기에 앞서 train안에 hold_d의 왜도가 크다고 볼수있다
최소1 에서 최대 1184 까지중 6만건이상이 hold_d가 < 100 이라고 볼수있다
target의 분포가 왜도가 심하다면 stacking, fold등 으로 적은데이터를 늘려줘야된다


2. 이상값 처리를 위해 RobustScaler를 진행하였으나, 학습 RMSE가 0.02, 0.03으로 과대적합이 발생했다. 따라서, L2규제를 적용하여 과대적합을 낮추었다.
XGBoost모델이나 LGBM 모델보다는 명목형 피쳐가 많은 경우 효과가 높은 모델인 CatBoost모델이 가장 정확도가 높았다.
회귀모델이므로 K-fold를 사용하였으며, n_splits는 10을 적용하였다.
CatBoost의 learning_rate은 0.03을 적용하였고, depth는 2로 설정하였다. 
